In [1]:
import newspaper as news
import requests
import scrapy
from urllib.parse import urlparse, parse_qs
from html.parser import HTMLParser

In [3]:
INITIAL_URL = "https://accounts.wsj.com/login?target=https%3A%2F%2Fwww.wsj.com%2F"
LOGIN_URL = "https://sso.accounts.dowjones.com/usernamepassword/login"
payload = {
    "username": "",  ## TODO: change to make username/password to read runtime input
    "password": "",
    "tenant": "sso"
}

## Initialize session
s = requests.Session()

In [4]:
"""
 WSJ log-in has some kind of hops. We need to go to INITIAL_URL to get redirected to 
 the dowjones universal log in page with a bunch of cookie info. Then we will use 
 the cookie info to post to the real log in form. This will return another callback
 page, that we will need to parse and read the content (in html) and post to a url
 again to actually log in to WSJ.
"""

# Getting the redirection here. Read the delicious cookie
init_r = s.get(INITIAL_URL)
params = parse_qs(urlparse(init_r.url).query)
for key, value in params.items():
    value = value[0]
    if key == "client": 
        payload[key + "_id"] = value
    else:
        payload[key] = value

In [5]:
# Post to the dowjones log-in page to get the call back
login_r = s.post(LOGIN_URL, data=payload)

In [7]:
# Parse the callback page to get formdata to post again
class CustomHTMLParser(HTMLParser):
    url = ''
    form_data = {}
    def handle_starttag(self, tag, attrs):
        if tag == "form":
            for key, value in attrs:
                if key == "action":
                    self.url = value
        
        if tag == "input":
            name = ""
            value = ""
            for key, val in attrs:
                if key == "name": 
                    name = val
                if key == "value":
                    value = val
            if name:
                self.form_data[name] = value

html_parser = CustomHTMLParser()
html_parser.feed(login_r.text)

In [8]:
# Post to the callback page to get a redirect link. Due to the double hop here we need to
# stop the first redirection and use a GET ourself
callback_r = s.post(html_parser.url, data=html_parser.form_data, allow_redirects=False)

# A quick check here is callback_r.status_code should return 302

# This time we will allow the automatic redirection. And... hooray! We are logged in
# What happens is that it hops and get another 302, and then redirect again to wsj.com 
# with a final 200 return
callback_r = s.get(callback_r.text[22:])